# Setting up the packages

Before running the code, please install the dependencies in a python environment. \n
Then install the package as "pip install -e ." when in the embryo_pipe main folder and in the environment. 

# Loading packages

In [ ]:
import embryo_pipe as ep
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Global parameters

In [2]:
# Image parameters
num_channels=4
ms2_channel=2
pp7_channel=1
time_per_image=54.3/60 #Frame Interval from ImageJ metadata
r_cutoff=2.5
g_cutoff=6

# Single embryo process

In [3]:
def process_embryo(path,outfolder,savestring='emb',time_per_image=54.3/60,frame_start=0,frame_stop=40, g_cutoff=6, r_cutoff=2.5,surv_thresh=5):
    """
    Automates the full analysis for a single embryo path.
    Returns aggregated metrics for that embryo.
    """
    # 1. Import and truncate images
    g_max, r_max = ep.import_image_czi(path)
    g_analyze = g_max[frame_start:frame_stop, :, :]
    r_analyze = r_max[frame_start:frame_stop, :, :]
    
    # 2. Nuclear segmentation and tracking
    # 'nuclei_track' and other functions must be defined in the notebook
    _, _, tracked_nucs = ep.nuclei_track(r_analyze, overlap_frac=0.5)
    _, tracked_nucs_filt = ep.nucs_border_time_filter(tracked_nucs, surv_thresh=surv_thresh)
    
    # 3. Foci segmentation
    g_foci, g_foci_int = ep.ms2_segmentation(g_analyze, tracked_nucs_filt, div_thresh=g_cutoff)
    r_foci, r_foci_int = ep.pp7_segmentation(r_analyze, tracked_nucs_filt, r_cutoff)
    
    # 4. Single nucleus intensity extraction
    # This returns dictionaries mapping nuc_id -> intensity_list
    g_ints, r_ints = ep.singlenuc_ints(tracked_nucs_filt, g_foci_int, r_foci_int)

    normed_tcourse,cum_tcourse,scyl_instexp,chrb_instexp, scyl_fociInt, chrb_fociInt=ep.normNucCount(g_ints,r_ints)

    fig,ax=plt.subplots(2,2,figsize=(8,8))

    ax[0,0].plot(time_per_image*np.arange(0,g_analyze.shape[0]),scyl_instexp,'go')
    ax[0,0].set_xlabel('Time (min)',fontsize=14)
    ax[0,0].set_ylabel('MS2 Intensity (au)',fontsize=14)
    ax[0,0].set_title('MS2 expression',fontsize=16)

    ax[0,1].plot(time_per_image*np.arange(0,r_analyze.shape[0]),chrb_instexp,'ro')
    ax[0,1].set_xlabel('Time (min)',fontsize=14)
    ax[0,1].set_ylabel('PP7 Intensity (au)',fontsize=14)
    ax[0,1].set_title('PP7 expression',fontsize=16)

    ax[1,0].plot(time_per_image*np.arange(0,g_analyze.shape[0]),scyl_fociInt,'go')
    ax[1,0].set_xlabel('Time (min)',fontsize=14)
    ax[1,0].set_ylabel('Foci intensity (au)',fontsize=14)
    ax[1,0].set_title('MS2 median intensity',fontsize=16)

    ax[1,1].plot(time_per_image*np.arange(0,r_analyze.shape[0]),chrb_fociInt,'ro')
    ax[1,1].set_xlabel('Time (min)',fontsize=14)
    ax[1,1].set_ylabel('Foci intensity (au))',fontsize=14)
    ax[1,1].set_title('PP7 median intensity',fontsize=16)
    fig.suptitle('Embryo: '+savestring,fontsize=14)
    fig.tight_layout()
    fig.savefig(os.path.join(outfolder,savestring+'.png'))
    plt.close()
    
    return g_ints, r_ints, normed_tcourse, cum_tcourse, scyl_instexp, chrb_instexp

## Storage variables

In [5]:
g_ints={}
r_ints={}
normed_tcourses={}
cum_tcourses={}
scyl_instexp={}
chrb_instexp={}

In [ ]:
def scyl_pos_lists(gburstInt_singlenuc,rburstInt_singelnuc):
    newkey=0
    rburstIntFilt={}
    for nuc in gburstInt_singlenuc.keys():
        g_ints=np.array(gburstInt_singlenuc[nuc])
        np.nan_to_num(g_ints,copy=False)
        if np.sum(g_ints>0):
            rburstIntFilt[newkey]=np.rint(rburstInt_singelnuc[nuc])
            newkey+=1
    return rburstIntFilt

## Embryos

In [ ]:
imagesfolder='imagefolder'
keys=['emb1','emb2','emb3','emb4','emb5','emb6']

# Defined manually from start to end of gastrulation

starts=[0,0,5,8,0,3,0] 
stops=[48,32,66,65,47,73,40]

for ctr in range(len(keys)):
    key=keys[ctr]
    g_ints[key], r_ints[key], normed_tcourses[key], cum_tcourses[key], scyl_instexp[key], chrb_instexp[key]=process_embryo(os.path.join(imagesfolder,key),os.path.join(imagesfolder,'panels'),savestring=key,frame_start=starts[ctr],frame_stop=stops[ctr])

/Users/pc2976/Documents/embryo_pipe/embryo_pipe/segmentation.py:52: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  foci_label=remove_small_objects(foci_label,min_size=2)


## Saving the Data

In [ ]:

dicts = [scyl_pos_lists(g_ints[key],r_ints[key]) for key in keys]

# Flatten lists and keep metadata
rows = []
for emb, d in zip(keys, dicts):
    for key, values in d.items():
        rows.append({
            'emb': emb,
            'values': values
        })

# Determine maximum list length across all rows
max_len = max(len(r['values']) for r in rows)

# Pad lists with NaN
for r in rows:
    pad_width = max_len - len(r['values'])
    if pad_width > 0:
        r['values'] = np.pad(
            r['values'],
            (pad_width, 0),
            mode='constant',
            constant_values=np.nan
        )

# Convert to DataFrame
values_df = pd.DataFrame(
    [r['values'] for r in rows],
    columns=[f'T{i+1}' for i in range(max_len)]
)

# Add metadata columns
meta_df = pd.DataFrame(
    [{'emb': r['emb']} for r in rows]
)

final_df = pd.concat([meta_df, values_df], axis=1)
final_df.to_csv('/Users/pc2976/DriveMirror/Levine lab/Group Meeting/Collab/RawBurstData/deltaCTCFDors_PP7BurstData.csv',sep='\t',index=False)

final_df

In [21]:
ctrl_embs=keys

normed_ctrl_lists = [list(normed_tcourses[x]) for x in ctrl_embs]
cum_ctrl_lists    = [list(cum_tcourses[x]) for x in ctrl_embs]

scyl_instexp_lists = [list(scyl_instexp[x]) for x in ctrl_embs]
chrb_instexp_lists = [list(chrb_instexp[x]) for x in ctrl_embs]

folder='/Users/pc2976/Documents/'

In [18]:
df_normed = ep.create_timeseries_dataframe(
    list_of_lists=normed_ctrl_lists,
    list_identifiers=ctrl_embs,
    time_per_image=time_per_image,
    total_time=0,
    output_path=os.path.join(folder, 'deltactcfDors_PP7normInstExp.csv'),
    column_prefix='ctrl'
)

# For cumulative data
df_cum = ep.create_timeseries_dataframe(
    list_of_lists=cum_ctrl_lists,
    list_identifiers=ctrl_embs,
    time_per_image=time_per_image,
    total_time=0,
    output_path=os.path.join(folder, 'deltactcfDors_PP7normCumExp.csv'),
    column_prefix='ctrl'
)

In [23]:
df_normed = ep.create_timeseries_dataframe(
    list_of_lists=scyl_instexp_lists,
    list_identifiers=ctrl_embs,
    time_per_image=time_per_image,
    total_time=0,
    output_path=os.path.join(folder, 'deltactcfDors_MS2TotInt.csv'),
    column_prefix='ctrl'
)

# For cumulative data
df_cum = ep.create_timeseries_dataframe(
    list_of_lists=chrb_instexp_lists,
    list_identifiers=ctrl_embs,
    time_per_image=time_per_image,
    total_time=0,
    output_path=os.path.join(folder, 'deltactcfDors_PP7TotInt.csv'),
    column_prefix='ctrl'
)